In [1]:
# imports and load the preproccessed data
import pandas as pd
from ast import literal_eval
from gensim import corpora, models
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models
import pyLDAvis
df = pd.read_csv("../preprocessed.csv")

In [2]:
# Convert the precossed_body into a list of token
df['tokens'] = df['preprocessed_body'].apply(literal_eval)

In [3]:
# Create the bag of words
dictionary = corpora.Dictionary(df['tokens'])
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=2000)
corpus = [dictionary.doc2bow(text) for text in df['tokens']]
bow_data = []
# Display the first 20 rows of BoW
for doc_idx, bow in enumerate(corpus[:20]):  
    word_freqs = [(dictionary[word_id], freq) for word_id, freq in bow]
    bow_data.append({'doc': doc_idx+1, 'words': word_freqs})
bow_df = pd.DataFrame(bow_data)
bow_df

,doc,words
0,1,"[('', 1), (another, 1), (cost, 1), (course, 1)..."
1,2,"[(course, 1), (thing, 1), (go, 1), (real, 1), ..."
2,3,"[(well, 1), (real, 1), (actually, 1), (almost,..."
3,4,"[(culture, 1), (like, 1), (lot, 1), (often, 1)..."
4,5,"[(probably, 1), (global, 1), (warm, 1), (due, ..."
5,6,"[('', 4), (like, 2), (might, 1), (need, 1), (s..."
6,7,"[(another, 1), (answer, 1), (car, 1), (inequal..."
7,8,"[(like, 1), (people, 1), (try, 1), (week, 1), ..."
8,9,"[(cost, 1), (go, 2), (far, 1), (addition, 1), ..."
9,10,"[(like, 1), (come, 1), (defend, 1), (force, 1)..."


In [4]:
# Create thhe LDA with 3 topics
lda_model = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=3,
    random_state=42,
    passes=10
)
# Print the topics
topics = lda_model.print_topics(-1, num_words=10)
for i, topic in topics:
    words = topic.replace('"', '').replace('*', '').replace('+', ',').split(',')
    clean_words = [w.strip() for w in words if w.strip()]
    print(f"Topic {i+1}: {', '.join(clean_words)}")

Topic 1: 0.022people, 0.014like, 0.014'', 0.013think, 0.012say, 0.011get, 0.011make, 0.010would, 0.010go, 0.010thing
Topic 2: 0.013year, 0.009global, 0.009would, 0.008gt, 0.008go, 0.007cause, 0.007warm, 0.006http, 0.006human, 0.006use
Topic 3: 0.035http, 0.022gt, 0.014-, 0.012amp, 0.012trump, 0.010government, 0.009country, 0.009policy, 0.008tax, 0.007''


In [5]:
# Display the topics by using pyLDAvis 
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.127557  0.115233       1        1  46.714948
1     -0.074822 -0.137155       2        1  31.272145
2      0.202379  0.021922       3        1  22.012907, topic_info=      Term          Freq         Total Category  logprob  loglift
348   http  17040.000000  17040.000000  Default  30.0000  30.0000
271     gt  16523.000000  16523.000000  Default  29.0000  29.0000
228    amp   4691.000000   4691.000000  Default  28.0000  28.0000
687  trump   4702.000000   4702.000000  Default  27.0000  27.0000
175      -   8066.000000   8066.000000  Default  26.0000  26.0000
..     ...           ...           ...      ...      ...      ...
505   work   1719.699784   5582.379824   Topic3  -5.4233   0.3361
30     say   1771.549582  13584.595541   Topic3  -5.3936  -0.5235
157   also   1538.089928   8266.672711   Topic3  -5.5349  -0.1681
340   want   1481.642500   6977.604820   Topic3  -5.5723  -0.0360
372  right   1465.055229   7435.532773   Topic3  -5.5835  -0.1108

[244 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         1  0.692336     ''
0         2  0.130591     ''
0         3  0.177025     ''
175       1  0.127189      -
175       2  0.199337      -
...     ...       ...    ...
225       1  0.207173   year
225       2  0.684125   year
225       3  0.108721   year
343       1  0.994639  youre
343       2  0.005143  youre

[375 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

In [6]:
# Evaluation metrics using Coherence score
coherence_model = CoherenceModel(model=lda_model, texts=df['tokens'], dictionary=dictionary, coherence='c_v')
coherence = coherence_model.get_coherence()
print(f"Coherence Score: {coherence:.4f}")

Coherence Score: 0.3941
